# Тестирование логгирования
_Воспроизведение экспериментов_

In [1]:
from datetime import datetime
from os.path import exists, join

import albumentations as A
import numpy as np
import pandas as pd
import torch
import torchvision
from torch.nn import CrossEntropyLoss
from torch.optim import SGD, Adam
from torch.optim.lr_scheduler import StepLR

from src.config import (MOT20_EXT_FIRST_AXIS_MEAN, MOT20_EXT_SECOND_AXIS_MEAN,
                        RESULTS_PATH)
from src.models.reidentification import SiameseBasicCNN, SiameseTransfered
from src.models.utils import resnet_grad_l4_unlock
from src.train import get_loaders, train_siamese
from src.train.loss import ContrastiveLoss
from src.train.utils import save_train_results
from src.transforms import get_norm_transform, get_resize_transform

/home/nick/reidentification/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nick/reidentification/.conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/nick/reidentification/.conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


## Создание и обработка csv

In [2]:
df = pd.DataFrame(columns=[
    'model_name',
    'datetime',
    'epoch_count',
    'optimizer',
    'lr',
    'gamma',
    'step_size',
    'loss_name',
    'dataset_config'
])

dt = str(datetime.now())

df.loc[len(df)] = [
    'test',
    dt,
    1,
    'adam',
    1e-3,
    0.5,
    2,
    'contr',
    'a'
]

df = df.astype({
    'model_name': 'object',
    'datetime': 'datetime64',
    'epoch_count': 'i4',
    'optimizer': 'object',
    'lr': 'float64',
    'gamma': 'float64',
    'step_size': 'i4',
    'loss_name': 'object',
    'dataset_config': 'object'
})

# df.astype({
#     'model_name': 'int32'
# })

df.dtypes

model_name                object
datetime          datetime64[ns]
epoch_count                int32
optimizer                 object
lr                       float64
gamma                    float64
step_size                  int32
loss_name                 object
dataset_config            object
dtype: object

In [3]:
file_path = join(RESULTS_PATH, 'experiments.csv')
config_path = join(RESULTS_PATH, 'configs.csv')

# # experiments.csv
# # pd.read_csv()
# df.to_csv(file_path, sep=',', index=False)

In [4]:
# pd.read_csv(file_path)

In [5]:
# if (not exists(file_path)):
#     # TODO: указать типы
#     pd.DataFrame(columns=[
#         'model_name',
#         'datetime',
#         'epoch_count',
#         'optimizer',
#         'lr',
#         'gamma',
#         'step_size',
#         'loss_name',
#         'dataset_config'
#     ]).to_csv(file_path, sep=',', index=False)

In [6]:
# df = pd.read_csv(file_path)

In [7]:
res = pd.DataFrame({
    'model_name': 'test2',
    'datetime': dt,
    'epoch_count': 1,
    'optimizer': 'adam',
    'lr': 1e-3,
    'gamma': 0.5,
    'step_size': 2,
    'loss_name': 'contr',
    'dataset_config': 'a'
}, index=[0])
res

,model_name,datetime,epoch_count,optimizer,lr,gamma,step_size,loss_name,dataset_config
0,test2,2023-05-15 14:13:24.784617,1,adam,0.001,0.5,2,contr,a


In [8]:
df = df.append(res)
df

/tmp/ipykernel_33766/3392409908.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(res)


,model_name,datetime,epoch_count,optimizer,lr,gamma,step_size,loss_name,dataset_config
0,test,2023-05-15 14:13:24.784617,1,adam,0.001,0.5,2,contr,a
0,test2,2023-05-15 14:13:24.784617,1,adam,0.001,0.5,2,contr,a


In [9]:
# df.to_csv(file_path, sep=',', index=False)

In [10]:
# pd.read_csv(file_path)

## Сохранение конфигов

In [11]:
df_config = pd.DataFrame(columns=[
    'dataset_config',
    'dataset',
    'dataset_use',
    'train_proportion',
    'val_proportion',
    'test_proportion',
    'batch_size',
])
df_config

,dataset_config,dataset,dataset_use,train_proportion,val_proportion,test_proportion,batch_size


In [12]:
res = pd.DataFrame({
    'dataset_config': 'mot1',
    'dataset': 'mot',
    'dataset_use': 0.02,
    'train_proportion': 0.7,
    'val_proportion': 0.15,
    'test_proportion': 0.15,
    'batch_size': 16
}, index=[0])

In [13]:
df_config = df_config.append(res)
df_config

/tmp/ipykernel_33766/76728779.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_config = df_config.append(res)


,dataset_config,dataset,dataset_use,train_proportion,val_proportion,test_proportion,batch_size
0,mot1,mot,0.02,0.7,0.15,0.15,16


In [14]:
'mot1' in df_config['dataset_config'].unique(), 'mot2' in df_config['dataset_config'].unique()

(True, False)

In [15]:
conf = {
    "mot20_ext_v1": {
        "dataset_config": 'mot20_ext_v1',
        "dataset": 'MOT20_ext',
        "dataset_use": 0.002,
        "train_proportion": 0.65,
        "val_proportion": 0.15,
        "test_proportion": 0.2,
        "batch_size": 16
    },
    "mot20_ext_v2": {
        "dataset_config": 'mot20_ext_v2',
        "dataset": 'MOT20_ext',
        "dataset_use": 0.002,
        "train_proportion": 0.65,
        "val_proportion": 0.15,
        "test_proportion": 0.2,
        "batch_size": 16,
        'extra_values': {
            'visibility_threshold': 0.7,
            'frame_distance': 3,
            'negative_proportion': 0.5
        }
    }
}

In [16]:
pd.DataFrame(conf['mot20_ext_v1'], index=[0])

,dataset_config,dataset,dataset_use,train_proportion,val_proportion,test_proportion,batch_size
0,mot20_ext_v1,MOT20_ext,0.002,0.65,0.15,0.2,16


## Проверка функций

In [17]:
# save_train_results(
#     'siamese1',
#     datetime.now(),
#     10, 
#     1e-3,
#     'adam',
#     'contrastive',
#     'mot20_ext_v1',
#     0.5,
#     3
# )

In [18]:
# save_train_results(
#     'siamese2',
#     datetime.now(),
#     10, 
#     1e-3,
#     'adam',
#     'contrastive',
#     'mot20_ext_v1',
#     0.5,
#     3,
#     conf['mot20_ext_v1']
# )

In [19]:
# save_train_results(
#     'siamese2',
#     datetime.now(),
#     10, 
#     1e-3,
#     'adam',
#     'contrastive',
#     'mot20_ext_v1',
#     0.5,
#     3,
#     conf['mot20_ext_v1']
# )

In [20]:
# save_train_results(
#     'siamese2',
#     datetime.now(),
#     10, 
#     1e-3,
#     'adam',
#     'contrastive',
#     'mot20_ext_v1',
#     0.5,
#     3,
#     conf['mot20_ext_v2']
# )

### Экстра параметры

In [21]:
extra = {
    'visibility_threshold': 0.7,
    'frame_distance': 3,
    'negative_proportion': 0.5
}

for k, v in zip(extra, extra.values()):
    print(k, v)

visibility_threshold 0.7
frame_distance 3
negative_proportion 0.5


In [22]:
';'.join([f'{k}={v}' for k, v in zip(extra, extra.values())])

'visibility_threshold=0.7;frame_distance=3;negative_proportion=0.5'

In [23]:
c = conf['mot20_ext_v2']
c['extra_values'] = ';'.join([f'{k}={v}' for k, v in zip(
    c['extra_values'], c['extra_values'].values())])
c

{'dataset_config': 'mot20_ext_v2',
 'dataset': 'MOT20_ext',
 'dataset_use': 0.002,
 'train_proportion': 0.65,
 'val_proportion': 0.15,
 'test_proportion': 0.2,
 'batch_size': 16,
 'extra_values': 'visibility_threshold=0.7;frame_distance=3;negative_proportion=0.5'}

In [24]:
# save_train_results(
#     'siamese2',
#     datetime.now(),
#     10, 
#     1e-3,
#     'adam',
#     'contrastive',
#     'mot20_ext_v1',
#     0.5,
#     3,
#     conf['mot20_ext_v2']
# )

In [25]:
# pd.read_csv(join(RESULTS_PATH, 'configs.csv'))

In [26]:
a = [1.1, 1.2, 1.3]
';'.join(map(str, a))

'1.1;1.2;1.3'

## Тестирование при обучении

In [27]:
# настройка параметров датасета
dataset_config = {
    'dataset_config': 'mot20_ext-v1a',
    'dataset': 'mot20_ext',
    'dataset_use': 0.00002,
    'val_proportion': 0.1,
    'test_proportion': 0.2,
    'batch_size': 1,
    'extra_parameters': {
        'visibility_threshold': 0.9,
        'negative_proportion': 0.4
    }
}

In [28]:
# создание трансформаций
resize_transform = get_resize_transform((MOT20_EXT_FIRST_AXIS_MEAN, MOT20_EXT_SECOND_AXIS_MEAN)) 
norm_transform = get_norm_transform()
transform = A.Compose([resize_transform, norm_transform])
# загрузка даталоадеров
train_loader, val_loader, test_loader = get_loaders(dataset_config, transform=transform)
# настройка модели
model_rn18_l4 = SiameseTransfered(
    # выделение признаков - resnet18
    torchvision.models.resnet18(pretrained=True),
    # разморозка layer4 и выходного линейного
    freeze_grad_fn=resnet_grad_l4_unlock,
    # имя
    name='siamese_resnet18_l4'
)

lr = 1e-3
criterion = ContrastiveLoss()
optimizer = Adam(model_rn18_l4.parameters(), lr)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


/home/nick/reidentification/.conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/nick/reidentification/.conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [29]:
model, results = train_siamese(
    config=dataset_config,
    criterion=criterion,
    device=device,
    epoch_count=1,
    lr=lr,
    model=model_rn18_l4,
    optimizer=optimizer,
    threshold=0.25,
    train_loader=train_loader,
    val_loader=val_loader,
)

Epoch 0/0:


100%|██████████| 6/6 [00:03<00:00,  1.81it/s]

train Loss: 14.5875 Acc: 0.0000



100%|██████████| 1/1 [00:00<00:00,  2.97it/s]

val Loss: 1219.5216 Acc: 0.0000
Model saved at siamese_resnet18_l4.pth



/home/nick/reidentification/src/train/utils.py:111: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({
/home/nick/reidentification/src/train/utils.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_config = df_config.append(config_append)


In [30]:
torch.eq(torch.tensor([1, 2]), torch.tensor([1, 1])).float().mean().item()

0.5

## Воспроизведение результатов

In [31]:
pd.read_csv(file_path)

,model_name,datetime,epoch_count,optimizer,lr,gamma,step_size,loss_name,dataset_config,extra_parameters,val_losses,val_accuracies,test_accuracy
0,siamese_resnet18_l4,2023-05-15 14:13:31.457448,1,Adam,0.001,NaN,NaN,ContrastiveLoss,mot20_ext-v1a,visibility_threshold=0.9;negative_proportion=0.4,1219.521606,0.0,NaN


In [32]:
pd.read_csv(config_path)

,dataset_config,dataset,dataset_use,train_proportion,val_proportion,test_proportion,batch_size,extra_parameters
0,mot20_ext-v1a,mot20_ext,0.00002,NaN,0.1,0.2,1,visibility_threshold=0.9;negative_proportion=0.4
